In [1]:
import torch
from torchvision import datasets, transforms
from tqdm import tqdm
import time

import sys
sys.path.append('C:/Users/SJ/Documents/FER-on-Exynos-device/Quantization/models')

from resnet50_PTSQ import resnet50

In [2]:
transforms = {
  'test' : transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [3]:
data_dir = 'C:/Users/SJ/Documents/FER-on-Exynos-device/FER/dataset'

test_set = datasets.ImageFolder(data_dir + '/test',
                                transforms['test'])
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle=False)

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available!')

cuda:0 is available!


In [5]:
weights_path = '../FER/resnet50.pt'
model = resnet50(pretrained=False).to(device)
model.load_state_dict(torch.load(weights_path))

<All keys matched successfully>

In [6]:
correct = 0
total = 0

# measure inference time -- better than time.time()
starter = torch.cuda.Event(enable_timing=True)
ender = torch.cuda.Event(enable_timing=True)

model.eval()

print('start predict!!!')

with torch.no_grad(): 
  starter.record()
  for data in tqdm(test_loader):
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum().item() 
  ender.record()

torch.cuda.synchronize()

infer_time = starter.elapsed_time(ender)

print(f'accuracy of 10000 test images: {100*correct/total}%')
print(f'time cost : {infer_time/1000} s')

start predict!!!


100%|██████████| 898/898 [00:09<00:00, 93.65it/s] 

accuracy of 10000 test images: 60.30927835051546%
time cost : 9.5768447265625 s
